In [1]:
"""
Example script demonstrating the portfolio optimization and backtesting system.

This script shows how to:
1. Load data from CSV files
2. Create and configure optimization strategies
3. Run backtests
4. Compare strategy performance
5. Generate reports and visualizations
"""

import pandas as pd
import numpy as np
from datetime import datetime
import tempfile
import os
from pathlib import Path

# Import our modules
from optfolio.data.loader import DataLoader
from optfolio.data.validator import DataValidator
from optfolio.backtesting.engine import Backtester
from optfolio.strategies.base import StrategyFactory

In [2]:
data_dir = "price"

# Step 2: Load and validate data
print("\n1. Loading and validating data...")
data_loader = DataLoader(data_dir)

# Load all available data
prices = data_loader.load_prices()
returns = data_loader.get_returns()

print(f"   Loaded {len(prices.columns)} tickers")
print(f"   Date range: {prices.index.min()} to {prices.index.max()}")
print(f"   Total observations: {len(prices)}")

# Validate data
validator = DataValidator()
price_issues = validator.validate_price_data(prices)
returns_issues = validator.validate_returns_data(returns)

print(f"   Data quality score: {validator.get_data_quality_score(prices):.3f}")

if price_issues['warnings']:
    print(f"   Warnings: {len(price_issues['warnings'])}")
if price_issues['errors']:
    print(f"   Errors: {len(price_issues['errors'])}")


1. Loading and validating data...


Loading price data: 100%|██████████| 40/40 [00:00<00:00, 496.03it/s]

   Loaded 40 tickers
   Date range: 2015-08-24 00:00:00 to 2025-08-21 00:00:00
   Total observations: 2514
   Data quality score: 1.000
   Warnings: 1


In [3]:
print("\n2. Creating optimization strategies...")

strategies = [
    StrategyFactory.create('equal_weight'),
    StrategyFactory.create('mean_variance', objective="sortino_ratio"),
    StrategyFactory.create('mean_variance', objective="sharpe_ratio"),
    StrategyFactory.create('random_weight', distribution="dirichlet", seed=42),
    StrategyFactory.create('black_litterman', 
                            prior_method="market_cap", view_method="momentum")
]

strategy_names = ["Equal Weight", "Mean-Variance (Sortino)", "Mean-Variance (Sharpe)", "Random Weight", "Black-Litterman"]
for i, strategy in enumerate(strategies):
    strategy.name = strategy_names[i]
    print(f"   Created: {strategy}")


2. Creating optimization strategies...
   Created: Equal Weight Strategy (Equal Weight)
   Created: Mean-Variance (Sortino) Strategy (sortino_ratio)
   Created: Mean-Variance (Sharpe) Strategy (sharpe_ratio)
   Created: Random Weight Strategy (dirichlet)
   Created: Black-Litterman Strategy (market_cap priors, momentum views)


In [4]:
# Step 4: Set up backtesting
print("\n3. Setting up backtesting...")

backtester = Backtester(
    initial_capital=100000.0,
    risk_free_rate=0.02,
    transaction_costs=0.001
)

# Load data into backtester
backtester.load_data(data_loader)


3. Setting up backtesting...


Loading price data: 100%|██████████| 40/40 [00:00<00:00, 394.52it/s]

Loaded data for 40 tickers
Date range: 2015-08-24 00:00:00 to 2025-08-21 00:00:00
Total observations: 2514


In [5]:
# Step 5: Run backtests
print("\n4. Running backtests...")

# Define rebalancing schedule (every 3 months, 1st week, 1st day)
rebalance_freq = {"months": 3, "weeks": 1, "days": 1}

# Run backtests for all strategies
results = backtester.run_multiple_backtests(
    strategies=strategies,
    rebalance_freq=rebalance_freq,
    start_date='2021-01-01',  # Start from 2021 to have enough history
    end_date='2023-12-31'
)

print(f"   Completed backtests for {len(results)} strategies")


4. Running backtests...
Running backtest for Equal Weight
Rebalancing dates: 9
Date range: 2021-01-03 00:00:00 to 2023-12-28 00:00:00
Running backtest for Mean-Variance (Sortino)
Rebalancing dates: 9
Date range: 2021-01-03 00:00:00 to 2023-12-28 00:00:00
Running backtest for Mean-Variance (Sharpe)
Rebalancing dates: 9
Date range: 2021-01-03 00:00:00 to 2023-12-28 00:00:00
Running backtest for Random Weight
Rebalancing dates: 9
Date range: 2021-01-03 00:00:00 to 2023-12-28 00:00:00
Running backtest for Black-Litterman
Rebalancing dates: 9
Date range: 2021-01-03 00:00:00 to 2023-12-28 00:00:00


/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:321: RuntimeWarning:

divide by zero encountered in matmul

/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:321: RuntimeWarning:

overflow encountered in matmul

/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:321: RuntimeWarning:

invalid value encountered in matmul

/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:347: RuntimeWarning:

divide by zero encountered in matmul

/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:347: RuntimeWarning:

overflow encountered in matmul

/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:347: RuntimeWarning:

invalid value encountered in matmul

/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:363: RuntimeWarning:

divide by zero encountered in matmul

/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:363: RuntimeWarnin

   Completed backtests for 5 strategies


In [9]:
strategy = StrategyFactory.create('black_litterman', 
                            prior_method="market_cap", view_method="momentum")
result = backtester.run_backtest(strategy=strategy,rebalance_freq=rebalance_freq,
    start_date='2021-01-01',  # Start from 2021 to have enough history
    end_date='2023-12-31')

Running backtest for black_litterman
Rebalancing dates: 9
Date range: 2021-01-03 00:00:00 to 2023-12-28 00:00:00


/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:280: RuntimeWarning:

divide by zero encountered in matmul

/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:280: RuntimeWarning:

overflow encountered in matmul

/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:280: RuntimeWarning:

invalid value encountered in matmul

/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:294: RuntimeWarning:

divide by zero encountered in matmul

/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:294: RuntimeWarning:

overflow encountered in matmul

/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:294: RuntimeWarning:

invalid value encountered in matmul

/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:293: RuntimeWarning:

divide by zero encountered in matmul

/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:293: RuntimeWarnin

In [ ]:
#!/usr/bin/env python3



def run_portfolio_analysis():
    """Run the main portfolio analysis."""
    print("=" * 60)
    print("PORTFOLIO OPTIMIZATION AND BACKTESTING SYSTEM")
    print("=" * 60)
    
    # Step 1: Create sample data
    # data_dir = create_sample_data()
    data_dir = "price"
    
    # Step 2: Load and validate data
    print("\n1. Loading and validating data...")
    data_loader = DataLoader(data_dir)
    
    # Load all available data
    prices = data_loader.load_prices()
    returns = data_loader.get_returns()
    
    print(f"   Loaded {len(prices.columns)} tickers")
    print(f"   Date range: {prices.index.min()} to {prices.index.max()}")
    print(f"   Total observations: {len(prices)}")
    
    # Validate data
    validator = DataValidator()
    price_issues = validator.validate_price_data(prices)
    returns_issues = validator.validate_returns_data(returns)
    
    print(f"   Data quality score: {validator.get_data_quality_score(prices):.3f}")
    
    if price_issues['warnings']:
        print(f"   Warnings: {len(price_issues['warnings'])}")
    if price_issues['errors']:
        print(f"   Errors: {len(price_issues['errors'])}")
    
    # Step 3: Create optimization strategies
    print("\n2. Creating optimization strategies...")
    
    strategies = [
        StrategyFactory.create('equal_weight'),
        StrategyFactory.create('mean_variance', objective="sortino_ratio"),
        StrategyFactory.create('mean_variance', objective="sharpe_ratio"),
        StrategyFactory.create('random_weight', distribution="dirichlet", seed=42),
        StrategyFactory.create('black_litterman', 
                             prior_method="market_cap", view_method="momentum")
    ]
    
    strategy_names = ["Equal Weight", "Mean-Variance (Sortino)", "Mean-Variance (Sharpe)", "Random Weight", "Black-Litterman"]
    for i, strategy in enumerate(strategies):
        strategy.name = strategy_names[i]
        print(f"   Created: {strategy}")
    
    # Step 4: Set up backtesting
    print("\n3. Setting up backtesting...")
    
    backtester = Backtester(
        initial_capital=100000.0,
        risk_free_rate=0.02,
        transaction_costs=0.001
    )
    
    # Load data into backtester
    backtester.load_data(data_loader)
    
    # Step 5: Run backtests
    print("\n4. Running backtests...")
    
    # Define rebalancing schedule (every 3 months, 1st week, 1st day)
    rebalance_freq = {"months": 3, "weeks": 1, "days": 1}
    
    # Run backtests for all strategies
    results = backtester.run_multiple_backtests(
        strategies=strategies,
        rebalance_freq=rebalance_freq,
        start_date='2021-01-01',  # Start from 2021 to have enough history
        end_date='2023-12-31'
    )
    
    print(f"   Completed backtests for {len(results)} strategies")
    
    # Step 6: Compare strategies
    print("\n5. Comparing strategy performance...")
    
    comparison = backtester.compare_strategies()
    print("\nStrategy Performance Comparison:")
    print("-" * 80)
    print(comparison.round(3).to_string(index=False))
    
    # Step 7: Generate detailed analysis
    print("\n6. Detailed strategy analysis...")
    
    for strategy_name, result in results.items():
        metrics = result['performance_metrics']
        summary = result['summary']
        
        print(f"\n{strategy_name}:")
        print(f"  Total Return: {metrics.get('total_return', 0):.2%}")
        print(f"  Annualized Return: {metrics.get('annualized_return', 0):.2%}")
        print(f"  Volatility: {metrics.get('volatility', 0):.2%}")
        print(f"  Sharpe Ratio: {metrics.get('sharpe_ratio', 0):.3f}")
        print(f"  Sortino Ratio: {metrics.get('sortino_ratio', 0):.3f}")
        print(f"  Max Drawdown: {metrics.get('max_drawdown', 0):.2%}")
        print(f"  Number of Transactions: {summary.get('num_transactions', 0)}")
        print(f"  Total Transaction Costs: ${summary.get('total_transaction_costs', 0):.2f}")
    
    # Step 8: Export results
    print("\n7. Exporting results...")
    
    # Export comparison to CSV
    comparison.to_csv('strategy_comparison.csv', index=False)
    print("   Exported strategy comparison to 'strategy_comparison.csv'")
    
    # Export detailed results to JSON
    backtester.export_results('detailed_results.json', format='json')
    print("   Exported detailed results to 'detailed_results.json'")
    
    # Step 9: Generate visualizations
    print("\n8. Generating visualizations...")
    
    try:
        import matplotlib.pyplot as plt
        
        # Plot portfolio values
        plt.figure(figsize=(12, 8))
        for strategy_name, result in results.items():
            if 'portfolio_values' in result and len(result['portfolio_values']) > 0:
                portfolio_series = result['portfolio_values']
                plt.plot(portfolio_series.index, portfolio_series.values, 
                        label=strategy_name, linewidth=2)
        
        plt.title('Portfolio Values Over Time')
        plt.xlabel('Date')
        plt.ylabel('Portfolio Value ($)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig('portfolio_values.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print("   Saved portfolio values plot to 'portfolio_values.png'")
        
        # Plot rolling metrics
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        for i, (strategy_name, result) in enumerate(results.items()):
            rolling_metrics = result['rolling_metrics']
            
            if not rolling_metrics.empty:
                # Rolling Sharpe ratio
                axes[0, 0].plot(rolling_metrics.index, rolling_metrics['rolling_sharpe'], 
                               label=strategy_name, alpha=0.7)
                
                # Rolling volatility
                axes[0, 1].plot(rolling_metrics.index, rolling_metrics['rolling_volatility'], 
                               label=strategy_name, alpha=0.7)
                
                # Rolling return
                axes[1, 0].plot(rolling_metrics.index, rolling_metrics['rolling_return'], 
                               label=strategy_name, alpha=0.7)
                
                # Rolling drawdown
                axes[1, 1].plot(rolling_metrics.index, rolling_metrics['rolling_drawdown'], 
                               label=strategy_name, alpha=0.7)
        
        axes[0, 0].set_title('Rolling Sharpe Ratio')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        axes[0, 1].set_title('Rolling Volatility')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        axes[1, 0].set_title('Rolling Return')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        axes[1, 1].set_title('Rolling Drawdown')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        # plt.savefig('rolling_metrics.png', dpi=300, bbox_inches='tight')
        plt.show()
        plt.close()
        
        print("   Saved rolling metrics plot to 'rolling_metrics.png'")
        
    except ImportError:
        print("   Matplotlib not available, skipping visualizations")
    
    # Step 10: Summary
    print("\n" + "=" * 60)
    print("ANALYSIS COMPLETE")
    print("=" * 60)
    
    # Find best performing strategy
    if not comparison.empty and 'Sharpe Ratio' in comparison.columns:
        best_strategy = comparison.loc[comparison['Sharpe Ratio'].idxmax(), 'Strategy']
        best_sharpe = comparison['Sharpe Ratio'].max()
        
        print(f"\nBest performing strategy (by Sharpe ratio): {best_strategy}")
        print(f"Sharpe ratio: {best_sharpe:.3f}")
    else:
        print(f"\nNo successful backtests completed. Check the error messages above.")
    
    # Clean up temporary data
    import shutil
    shutil.rmtree(data_dir.parent)
    
    print(f"\nTemporary data cleaned up")
    print(f"\nGenerated files:")
    print(f"  - strategy_comparison.csv")
    print(f"  - detailed_results.json")
    print(f"  - portfolio_values.png")
    print(f"  - rolling_metrics.png")





In [10]:
"""Run the main portfolio analysis."""
print("=" * 60)
print("PORTFOLIO OPTIMIZATION AND BACKTESTING SYSTEM")
print("=" * 60)

# Step 1: Create sample data
# data_dir = create_sample_data()
data_dir = "price"

# Step 2: Load and validate data
print("\n1. Loading and validating data...")
data_loader = DataLoader(data_dir)

# Load all available data
prices = data_loader.load_prices()
returns = data_loader.get_returns()

print(f"   Loaded {len(prices.columns)} tickers")
print(f"   Date range: {prices.index.min()} to {prices.index.max()}")
print(f"   Total observations: {len(prices)}")

# Validate data
validator = DataValidator()
price_issues = validator.validate_price_data(prices)
returns_issues = validator.validate_returns_data(returns)

print(f"   Data quality score: {validator.get_data_quality_score(prices):.3f}")

if price_issues['warnings']:
    print(f"   Warnings: {len(price_issues['warnings'])}")
if price_issues['errors']:
    print(f"   Errors: {len(price_issues['errors'])}")

# Step 3: Create optimization strategies
print("\n2. Creating optimization strategies...")

strategies = [
    StrategyFactory.create('equal_weight'),
    StrategyFactory.create('mean_variance', objective="sortino_ratio"),
    StrategyFactory.create('mean_variance', objective="sharpe_ratio"),
    StrategyFactory.create('random_weight', distribution="dirichlet", seed=42),
    StrategyFactory.create('black_litterman', 
                            prior_method="market_cap", view_method="momentum")
]

strategy_names = ["Equal Weight", "Mean-Variance (Sortino)", "Mean-Variance (Sharpe)", "Random Weight", "Black-Litterman"]
for i, strategy in enumerate(strategies):
    strategy.name = strategy_names[i]
    print(f"   Created: {strategy}")

# Step 4: Set up backtesting
print("\n3. Setting up backtesting...")

backtester = Backtester(
    initial_capital=100000.0,
    risk_free_rate=0.02,
    transaction_costs=0.001
)

# Load data into backtester
backtester.load_data(data_loader)

# Step 5: Run backtests
print("\n4. Running backtests...")

# Define rebalancing schedule (every 3 months, 1st week, 1st day)
rebalance_freq = {"months": 3, "weeks": 1, "days": 1}

# Run backtests for all strategies
results = backtester.run_multiple_backtests(
    strategies=strategies,
    rebalance_freq=rebalance_freq,
    start_date='2021-01-01',  # Start from 2021 to have enough history
    end_date='2023-12-31'
)

print(f"   Completed backtests for {len(results)} strategies")

# Step 6: Compare strategies
print("\n5. Comparing strategy performance...")

comparison = backtester.compare_strategies()
print("\nStrategy Performance Comparison:")
print("-" * 80)
print(comparison.round(3).to_string(index=False))

# Step 7: Generate detailed analysis
print("\n6. Detailed strategy analysis...")

for strategy_name, result in results.items():
    metrics = result['performance_metrics']
    summary = result['summary']
    
    print(f"\n{strategy_name}:")
    print(f"  Total Return: {metrics.get('total_return', 0):.2%}")
    print(f"  Annualized Return: {metrics.get('annualized_return', 0):.2%}")
    print(f"  Volatility: {metrics.get('volatility', 0):.2%}")
    print(f"  Sharpe Ratio: {metrics.get('sharpe_ratio', 0):.3f}")
    print(f"  Sortino Ratio: {metrics.get('sortino_ratio', 0):.3f}")
    print(f"  Max Drawdown: {metrics.get('max_drawdown', 0):.2%}")
    print(f"  Number of Transactions: {summary.get('num_transactions', 0)}")
    print(f"  Total Transaction Costs: ${summary.get('total_transaction_costs', 0):.2f}")

# Step 8: Export results
print("\n7. Exporting results...")

PORTFOLIO OPTIMIZATION AND BACKTESTING SYSTEM

1. Loading and validating data...


Loading price data: 100%|██████████| 40/40 [00:00<00:00, 389.40it/s]


   Loaded 40 tickers
   Date range: 2015-08-24 00:00:00 to 2025-08-21 00:00:00
   Total observations: 2514
   Data quality score: 1.000
   Warnings: 1

2. Creating optimization strategies...
   Created: Equal Weight Strategy (Equal Weight)
   Created: Mean-Variance (Sortino) Strategy (sortino_ratio)
   Created: Mean-Variance (Sharpe) Strategy (sharpe_ratio)
   Created: Random Weight Strategy (dirichlet)
   Created: Black-Litterman Strategy (market_cap priors, momentum views)

3. Setting up backtesting...


Loading price data: 100%|██████████| 40/40 [00:00<00:00, 926.10it/s]

Loaded data for 40 tickers
Date range: 2015-08-24 00:00:00 to 2025-08-21 00:00:00
Total observations: 2514

4. Running backtests...
Running backtest for Equal Weight
Rebalancing dates: 9
Date range: 2021-01-03 00:00:00 to 2023-12-28 00:00:00
Error running backtest for Equal Weight: Length of values (49) does not match length of index (50)
Running backtest for Mean-Variance (Sortino)
Rebalancing dates: 9
Date range: 2021-01-03 00:00:00 to 2023-12-28 00:00:00



/Users/michael/projects/optfolio/optfolio/strategies/base.py:80: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



Error running backtest for Mean-Variance (Sortino): Length of values (49) does not match length of index (50)
Running backtest for Mean-Variance (Sharpe)
Rebalancing dates: 9
Date range: 2021-01-03 00:00:00 to 2023-12-28 00:00:00
Error running backtest for Mean-Variance (Sharpe): Length of values (49) does not match length of index (50)
Running backtest for Random Weight
Rebalancing dates: 9
Date range: 2021-01-03 00:00:00 to 2023-12-28 00:00:00
Error running backtest for Random Weight: Length of values (49) does not match length of index (50)
Running backtest for Black-Litterman
Rebalancing dates: 9
Date range: 2021-01-03 00:00:00 to 2023-12-28 00:00:00


/Users/michael/projects/optfolio/optfolio/strategies/base.py:80: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/Users/michael/projects/optfolio/optfolio/strategies/base.py:80: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:280: RuntimeWarning:

divide by zero encountered in matmul

/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:280: RuntimeWarning:

overflow encountered in matmul

/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:280: RuntimeWarning:

invalid value encountered in matmul

/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:294: RuntimeWarning:

divide by zero encountered in matmul

/Users/michael/projects/optfolio/optfolio/strategies/black_litterman.py:2

Error running backtest for Black-Litterman: Length of values (49) does not match length of index (50)
   Completed backtests for 0 strategies

5. Comparing strategy performance...

Strategy Performance Comparison:
--------------------------------------------------------------------------------
Empty DataFrame
Columns: []
Index: []

6. Detailed strategy analysis...

7. Exporting results...
